# Mi idea

We are going to solve the folloeing problem

$$
            \begin{dcases}
                \min_{p \geq 0, \textcolor{red}{\pi \in Y}} \sum_{m=1}^{M}\frac{1}{M}\sum_{r=1}^{R}\sum_{s=1}^{S^{m}} \| \xi_r - \zeta_{s}^{m}\|^2 \pi_{rs}^{m} \\
                \textbf{s.t }\sum_{r=1}^R \pi_{rs}^{(m)} = q_s^{(m)}, \quad 
                \sum_{s=1}^{S^{(m)}} \pi_{rs}^{(m)} = p_r 
            \end{dcases}
$$

With constrains such as: $Y = \{\pi \,\large| \,||\pi^{(m)}||_F \leq \tau\}$ or $\sum_{s} \pi_{rs} \leq c_r$.

So my idea is: As the transport plan are matrices we could solve the following problem:

$$
\begin{dcases}
    \min_{Y} \frac{1}{2} ||P - \pi ||_F^2\\

    \textbf{s.t }\sum_{r=1}^R P_{rs}^{(m)} = q_s^{(m)}, \quad 
    \sum_{s=1}^{S^{(m)}} P_{rs}^{(m)} = p_r
\end{dcases}
$$

That is we could find the projection of $\pi$ on $Y \cap \mathcal{U}(\nu^{m},\mu)$ where $\mathcal{U}(\nu^{m},\mu)$ is the set of couplings betewn $\nu^{m},\mu$.

## ADMM

We can reformulate the problem as follows:

$$
\begin{dcases}
    \min_{P,Z} \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) \\   

    \textbf{s.t } P - Z = 0
\end{dcases}
$$

This leads to tink on the ADMM algo. The augmented lagrangian is:

$$
    \mathcal{L}_{\rho}(P,Z,Y) = \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) + \langle Y, P-Z \rangle + \frac{\rho}{2}||P-Z||_F^2
$$

To simplify this we see that:

$$
\frac{\rho}{2} ||X + \frac{Y}{\rho}||^2 - \frac{1}{2\rho}||Y||^2 = \frac{\rho}{2}\big( ||X||^2 + 2\langle X, \frac{Y}{\rho} \rangle + ||\frac{Y}{\rho}||^2\big)-\frac{1}{2\rho}||Y||^2\\
=\frac{\rho}{2}||X||^2 + \langle X, Y \rangle 
$$

If $X = P-Z$: 

$$
\langle P-Z, Y \rangle + \frac{\rho}{2}||P-Z||^2 =\frac{\rho}{2} ||P - Z + \frac{Y}{\rho}||^2 -\frac{1}{2}||\frac{Y}{\rho}||^2
$$

If we do $U = \frac{Y}{\rho}$ we get the new lagrangian:

$$
    \mathcal{L}_{\rho}(P,Z,U) = \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) + \frac{\rho}{2} ||P - Z + U||^2 -\frac{1}{2}||U||^2
$$

So we propoce the following iterations:

$$
P^{k+1} = \argmin_{P \in \mathcal{U}(\nu^{m},\mu)}  \frac{1}{2} ||P - \pi ||_F^2  + \frac{\rho}{2} ||P - (Z^K - U^K)||^2\\

Z^{k+1} = \argmin_{Z \in \mathcal{B}_\tau}  \frac{\rho}{2} ||P^{k+1} - Z + U^k||^2\\

U^{k+1} = U^k + P^{k+1} - Z^{k+1}
$$

We see that, for instance:

$$
Z^{k+1} = \argmin_{Z \in \mathcal{B}_\tau}  \frac{\rho}{2} ||P^{k+1} + U^k  - Z||^2 = \mathcal{P}_{\mathcal{B}_\tau}(P^{k+1} + U^k) = \begin{cases}
                            P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F \leq \tau\\
                            \frac{\tau}{||P^{k+1} + U^k||_F} P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F > \tau        
                        \end{cases}
\\
$$

if we take a look onto the first actualization:

$$
\frac{1}{2} ||P - \pi ||_F^2  + \frac{\rho}{2} ||P - (Z^K - U^K)||^2 = \frac{1}{2}||P||^2 - \langle P,\pi \rangle + \frac{1}{2}||\pi||^2 + \frac{\rho}{2}\big( ||P||^2 - 2\langle P, Z^k - U^k \rangle + ||Z^k - U^k||^2\big)\\
= \frac{1}{2}||P||^2 - \langle P,\pi \rangle + \frac{\rho}{2}||P||^2 - \rho\langle P, Z^k - U^k \rangle\\\
= \frac{1+\rho}{2}||P||^2 - \langle P,\pi + \rho(Z^k - U^k) \rangle
$$

We want to complete the square so:

$$
\frac{1+\rho}{2}||P-M||^2 = \frac{1+\rho}{2}||P||^2 - \langle P,\pi + \rho(Z^k - U^k) \rangle\\
= \frac{1+\rho}{2}||P||^2 - (1+\rho)\langle P,M \rangle + \frac{1+\rho}{2}||M||^2
$$

If $M$ does not depend on $P$ we can ignore it on the minimization:

$$
(1+\rho)\langle P,M \rangle = \langle P,\pi + \rho(Z^k - U^k) \rangle\\
M = \frac{\pi + \rho(Z^k - U^k)}{1+\rho}
$$

So:
$$
P^{k+1} = \argmin_{P \in \mathcal{U}(\nu^{m},\mu)}  \frac{1+\rho}{2}||P-\frac{\pi + \rho(Z^k - U^k)}{1+\rho}||^2 = \mathcal{P}_{\mathcal{U}(\nu^{m},\mu)} \big(\frac{\pi + \rho(Z^k - U^k)}{1+\rho} \big)\\
$$

Finally we have:

$$
Z^{k+1} = \begin{cases}
            P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F \leq \tau\\
            \frac{\tau}{||P^{k+1} + U^k||_F} P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F > \tau        
        \end{cases}\\

P^{k+1} = \mathcal{P}_{\mathcal{U}(\nu^{m},\mu)} \big(\frac{\pi + \rho(Z^k - U^k)}{1+\rho} \big)\\
U^{k+1} = U^k + P^{k+1} - Z^{k+1}

$$

The only problem is solving the update of $P$, lets suppose that projection exists (PROVE), we can use Dikstra algorihm for this as the set $\mathcal{U}(\nu^{m},\mu) = \{P : P1 = \nu\} \cap \{P : P^{T}1 = \mu\} \cap \{P : P \geq 0\}$ each one of the is convex, we will name them $C_1, C_2, C_3$.

We can see (PROVE) the the projection of a matrix $Q \in \mathbb{R}^{m\times n}$ on each of the previous sets are:

$$
\mathcal{P}_{C_1}(Q) = Q + \frac{\nu^m - Q1}{n}1^{T}\\
\mathcal{P}_{C_2}(Q) = Q + 1\frac{\mu - Q^T1^T}{m}\\
\mathcal{P}_{C_3}(Q) = \max\{Q,0\}
$$

So, Dikstra is as follows:

Let $P^0 = A$ (matrix to project), $R_1 = R_2 = R_3 = 0$ and:

$$
Q_1 = P^{K} + R_1^{K}\\
P_1 = \mathcal{P}_{C_1}(Q_1)\\
R_1^{k+1} = Q_1 - P_1\\
Q_2 = P_1 + R_2^{k}\\
P_2 = \mathcal{P}_{C_2}(Q_2)\\
R_2^{k+1} = Q_2 - P_2\\
Q_3 = P_2 + R_3^{k}\\
P^{k+1} = \mathcal{P}_{C_3}(Q_3)\\
R_3^{k+1} = Q_3 - P^{k+1}\\
$$

